In [39]:
import numpy as np
from scipy.stats import entropy
from math import log, e
import pandas as pd
import math

In [74]:
medecin_gen = pd.read_csv('medecin_gen.csv')

In [41]:
def calc_entropy(column):
    counts = np.bincount(column)
    probability = counts / len(column)
    entropy = 0
    for prob in probability:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    return -entropy

In [24]:
medecin_gen.drop(columns= ['Unnamed: 0'],inplace=True)

In [54]:
medecin_gen.columns

Index(['Doctor Name', 'Specialty', 'card_0', 'card_3', 'card_4', 'Latitude',
       'Longitude', 'Skills Processed', 'RPPS', 'Nb Skills', 'moyens_paiement',
       'carte_vitale', 'secteur', 'Présentation', 'Langues parlées',
       'Diplômes nationaux et universitaires', 'Autres formations',
       'Expériences', 'Travaux et publications', 'Prix et distinctions',
       'site', 'Site web', 'Formations',
       'Diplômes nationaux et universitaires b', 'Nb caractères présentation',
       'Autres formations b', 'Nb langues', 'Expériences b', 'Carac_card_1',
       'Transport', 'Parking', 'Infos', 'Coordonnées', 'Sans RDV',
       'nb terms card_0', 'nb terms card_3', 'nb terms card_4',
       'nb carac card_0', 'nb carac card_3', 'nb carac card_4'],
      dtype='object')

In [76]:
liste_var = ['Nb Skills', 'moyens_paiement',
       'carte_vitale', 'secteur',
       'site',
       'Diplômes nationaux et universitaires b', 'Nb caractères présentation',
       'Autres formations b','Travaux et publications b','Nb langues', 'Expériences b', 'Carac_card_1',
       'Transport', 'Parking', 'Infos', 'Coordonnées', 'Sans RDV', 'Formations b',
       'nb terms card_0', 'nb terms card_3', 'nb terms card_4',
       'nb carac card_0', 'nb carac card_3', 'nb carac card_4']
entropy={}
for nom_colonne in liste_var : 
    entropy[nom_colonne]=calc_entropy(medecin_gen[nom_colonne])
print(entropy)

{'Nb Skills': 1.9416470146081055, 'moyens_paiement': 0.17203694935311378, 'carte_vitale': 0.17203694935311378, 'secteur': 1.1401156785146092, 'site': 0.7793498372920852, 'Diplômes nationaux et universitaires b': 0.9881108365218301, 'Nb caractères présentation': 5.234120167580196, 'Autres formations b': 0.8904916402194913, 'Travaux et publications b': 0.39124356362925566, 'Nb langues': 1.9516536723433617, 'Expériences b': 0.6193821946787638, 'Carac_card_1': 5.080274013734042, 'Transport': 0.17203694935311378, 'Parking': 0.9881108365218301, 'Infos': -0.0, 'Coordonnées': 0.9881108365218301, 'Sans RDV': -0.0, 'Formations b': 0.9182958340544896, 'nb terms card_0': 2.955232570939141, 'nb terms card_3': 2.40262695595726, 'nb terms card_4': 3.420554630318644, 'nb carac card_0': 2.955232570939141, 'nb carac card_3': 2.40262695595726, 'nb carac card_4': 3.420554630318644}
